In [1]:
from datetime import datetime, timedelta

import pandas as pd
import xarray as xr

In [2]:
study_timeperiod = pd.date_range(start="2014-09", end="2017-04", freq="1M")
print(f"Start: {study_timeperiod[0]}, End: {study_timeperiod[-1]}")

Start: 2014-09-30 00:00:00, End: 2017-03-31 00:00:00


# Construct inputs datasets

## MOZART

In [3]:
file_paths = []
for month in study_timeperiod:
    yyyy, mm, _ = str(month).split("-")
    file_paths.append(
        f"../1_transport/intermediates/MOZART/output/BasisFnsUpdated/{yyyy}{mm}/"
        f"BasisFnsUpdated.mz4.h0.{yyyy}-{mm}-01-03600.nc"
    )


In [4]:
def round_nearest_hour(t):
    # round input datetime to nearest hour by adding a timedelta hour if minute >= 30
    return t.replace(second=0, microsecond=0, minute=0, hour=t.hour) + timedelta(
        hours=t.minute // 30
    )

def from_gregorian(t):
    # NOTE: int() gives floor which is what we want here
    date_time = datetime.fromordinal(int(t)) + timedelta(days=t % 1)
    return round_nearest_hour(date_time)


def prepare_mozart_file(ds):
    # format gregorian ordinal time as datetime
    ds["time"] = pd.to_datetime([from_gregorian(t) for t in ds.time.values])
    # extract co2 emissions array
    return ds["CO2_SRF_EMIS_avrg"]


# concatenate CO2_SRF_EMIS_avrg variable from MOZART diagnostics
with xr.open_mfdataset(file_paths, preprocess=prepare_mozart_file, decode_times=False) as ds:
    # for some reason, the files are off by a year
    ds["time"] = ds["time"].to_series() - pd.DateOffset(years=1)
    da_mozart = ds["CO2_SRF_EMIS_avrg"]

# one observation for every hour of every day in study period, in units kg/m^2/s
da_mozart

<xarray.DataArray 'CO2_SRF_EMIS_avrg' (time: 22632, lat: 96, lon: 144)>
dask.array<concatenate, shape=(22632, 96, 144), dtype=float32, chunksize=(744, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2014-09-01T01:00:00 ... 2017-04-01
Attributes:
    units:    KG/M^2/S

In [5]:
da_mozart.to_netcdf("../data/mozart_inputs.nc")

In [6]:
# define end of study period
mozart_datetime_end = da_mozart.time.values.max()

## GEOS Chem

In [7]:
def extract_co2_emission_geoschem(ds):
    return ds["EmisCO2_Total"]


geos_chem_path_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/HEMCO_diagnostics.*.nc"
)
with xr.open_mfdataset(geos_chem_path_glob, preprocess=extract_co2_emission_geoschem) as ds:
    # compute the sum over the pressure levels
    da_geoschem = ds["EmisCO2_Total"].sum(dim=["lev"])

    # clip to available mozart date range
    da_geoschem = da_geoschem.where(da_geoschem["time"] <= mozart_datetime_end, drop=True)

# one observation for every hour of every day in study period, in units kg/m^2/s
da_geoschem


<xarray.DataArray 'EmisCO2_Total' (time: 22632, lat: 91, lon: 144)>
dask.array<where, shape=(22632, 91, 144), dtype=float32, chunksize=(744, 91, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-03-31T23:30:00
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5

In [8]:
da_geoschem.to_netcdf("../data/geoschem_inputs.nc")

# Construct outputs datasets